In [1]:
import os
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import os
import open3d as o3d

def set_mesh_color(mesh, color):
    if not mesh.has_vertices():
        print("Mesh has no vertices to color.")
        return
    colors = [color for _ in range(len(mesh.vertices))]
    mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

def process_point_cloud(ply_file, output_mesh_file, color=(0.1, 0.1, 0.7)):
    # Load point cloud
    pcd = o3d.io.read_point_cloud(ply_file)
    
    # Downsample
    pcd = pcd.voxel_down_sample(voxel_size=0.02)
    
    # Noise removal
    pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    
    # Estimate normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    # Surface reconstruction using Poisson method
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
    
    # Check if the mesh has vertices
    if not mesh.has_vertices():
        print(f"No vertices found in the generated mesh for {ply_file}. Skipping.")
        return
    
    mesh.compute_vertex_normals()
    
    # Set the mesh color
    set_mesh_color(mesh, color)
    
    # Save the mesh
    o3d.io.write_triangle_mesh(output_mesh_file, mesh, write_vertex_normals=True, write_vertex_colors=True)
    
    print(f"Processed and saved: {output_mesh_file}")

def visualize_mesh(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()
    
    if not mesh.has_vertex_colors():
        print(f"Mesh {mesh_file} has no vertex colors. Visualizing without colors.")
    
    # Ensure the vertex colors are used in visualization
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

# Function to compute properties of the mesh
def compute_properties(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()

    # Surface area
    surface_area = mesh.get_surface_area()

    return surface_area

# Directory paths
input_ply_dir = r"D:\ply_final\two"
output_mesh_dir = r"D:\poisson"

# Ensure output directory exists
os.makedirs(output_mesh_dir, exist_ok=True)

# Process each point cloud
for ply_file in os.listdir(input_ply_dir):
    if ply_file.endswith(".ply"):
        ply_file_path = os.path.join(input_ply_dir, ply_file)
        output_mesh_file = os.path.join(output_mesh_dir, ply_file.replace(".ply", "_mesh.obj"))
        
        process_point_cloud(ply_file_path, output_mesh_file, color=(0.4, 0.4, 0.4))  # Change color as needed
        #visualize_mesh(output_mesh_file)
        
        surface_area = compute_properties(output_mesh_file)
        print(f"Mesh {output_mesh_file}: Surface Area = {surface_area}")


[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\poisson\point_cloud_1000_mesh.obj
Mesh D:\poisson\point_cloud_1000_mesh.obj: Surface Area = 514409.1273788803
[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\poisson\point_cloud_501_mesh.obj
Mesh D:\poisson\point_cloud_501_mesh.obj: Surface Area = 610071.8000943839
[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\poisson\point_cloud_502_mesh.obj
Mesh D:\poisson\point_cloud_502_mesh.obj: Surface Area = 505089.7778604501
[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\poisson\point_cloud_503_mesh.obj
Mesh D:\poisson\point_cloud_503_mesh.obj: Surface Area = 519742.76791014516
[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\poisson\point_cloud_504_mesh.obj
Mesh D:\poisson\point_cloud_504_mesh.obj: Surface Area = 504311.75560907344
[Open3D WARNING] Write OBJ can 